### Utilites 

In [114]:
import psycopg2
from psycopg2 import sql
import csv

In [112]:
db_params_default = {
    'host': "desperado-db.ctldmj6kaxoc.us-east-2.rds.amazonaws.com",
    'port': '5432',
    'user': 'desperado',
    'password': '6156dbdesperado',
    'database': 'postgres'
}

In [115]:
# Connect to the PostgreSQL server
with psycopg2.connect(**db_params_default) as connection:
    # Create a cursor
    with connection.cursor() as cursor:
        # Execute the query to list databases
        cursor.execute("SELECT datname FROM pg_database;")

        # Fetch all rows
        databases = cursor.fetchall()

# Print the list of databases
print("List of databases:")
for db in databases:
    print(db[0])

List of databases:
template0
template1
postgres
rdsadmin
application
userinfo


### 1. Userinfo Database Related

In [107]:
# Connect to the existing database
with psycopg2.connect(**db_params) as connection:
    # Get the current isolation level
    isolation_level = connection.isolation_level

print(f"Current isolation level: {isolation_level}")

Current isolation level: None


In [116]:
# db parameters for the userinfo database
db_params_userinfo = {
    'host': "desperado-db.ctldmj6kaxoc.us-east-2.rds.amazonaws.com",
    'port': '5432',
    'user': 'desperado',
    'password': '6156dbdesperado',
    'database': 'userinfo'
}

In [17]:
# delete all entries
delete_query = sql.SQL("DELETE FROM {}").format(sql.Identifier("userinfo"))

cursor = connection.cursor()
# Execute the query
cursor.execute(delete_query)
count_after_deletion = cursor.fetchone()[0]
print(f"Number of entries in userinfo after deletion: {count_after_deletion}")
# Commit the changes
conn.commit()

ProgrammingError: no results to fetch

In [119]:
# Define the table schema
table_schema = {
    'uuid': 'VARCHAR(32) PRIMARY KEY',
    'name': 'VARCHAR(255) NOT NULL',
    'company': 'VARCHAR(255)',
    'email': 'VARCHAR(255) NOT NULL',
    'is_recruiter': 'BOOLEAN NOT NULL',
    'picture_url' :'VARCHAR(255) NULL'
}
# Establish a connection to the PostgreSQL database
with psycopg2.connect(**db_params_userinfo) as conn_app:
    with conn_app.cursor() as cursor:
        # Create the table using the specified schema
        create_table_query = f"CREATE TABLE IF NOT EXISTS userinfo ({', '.join(f'{col} {data_type}' for col, data_type in table_schema.items())});"
        cursor.execute(create_table_query)

    # Commit the changes
    conn_app.commit()

print("Table 'userinfo' created successfully.")

Table 'userinfo' created successfully.


In [122]:
# populate tabl with the csv file
csv_file_path = './userinfo.csv'

# Establish a connection to the PostgreSQL database
with psycopg2.connect(**db_params_userinfo) as conn:
    with conn.cursor() as cursor:
        # Read the CSV file
        with open(csv_file_path, 'r') as csvfile:
            csv_reader = csv.DictReader(csvfile)
            for row in csv_reader:                
                # Prepare the SQL query for insertion
                insert_query = f"INSERT INTO userinfo ({', '.join(row.keys())}) VALUES ({', '.join(['%s'] * len(row))});"

                # Execute the query
                cursor.execute(insert_query, tuple(row.values()))

    # Commit the changes
    conn.commit()

print("Data inserted successfully.")

Data inserted successfully.


In [123]:
# sanity check - list 10 entries
with psycopg2.connect(**db_params_userinfo) as conn:
    with conn.cursor() as cursor:
        cursor.execute("SELECT * FROM userinfo LIMIT 10;")
        rows = cursor.fetchall()
        
        # Process the results
        if rows:
            print("Data in the table:")
            for row in rows:
                print(row)
        else:
            print("No data found in the table.")

Data in the table:
('6562d26afc13ae6bdf4cd82e', 'Shawnee Simanenko', 'Edgeclub', 'ssimanenko0@smugmug.com', False, 'https://robohash.org/etexercitationemprovident.png?size=50x50&set=set1')
('6562d26afc13ae6bdf4cd82f', 'Ibby Huggons', 'Dynabox', 'ihuggons1@simplemachines.org', True, 'https://robohash.org/liberodelectusut.png?size=50x50&set=set1')
('6562d26afc13ae6bdf4cd830', 'Granville Menelaws', 'Fadeo', 'gmenelaws2@army.mil', False, 'https://robohash.org/pariaturdoloremullam.png?size=50x50&set=set1')
('6562d26afc13ae6bdf4cd831', 'Cly Janowski', 'Avamba', 'cjanowski3@fotki.com', True, 'https://robohash.org/utetomnis.png?size=50x50&set=set1')
('6562d26afc13ae6bdf4cd832', 'Sarajane Shutle', 'Mybuzz', 'sshutle4@ow.ly', False, 'https://robohash.org/aeased.png?size=50x50&set=set1')
('6562d26afc13ae6bdf4cd833', 'Harmon Younger', 'Roodel', 'hyounger5@yolasite.com', True, 'https://robohash.org/velomnisneque.png?size=50x50&set=set1')
('6562d26afc13ae6bdf4cd834', 'Mariele Tennison', 'Linklinks',

### 2. Jobs 

### 3. Application Database Related

In [10]:
# create new database in the same RDS instance
new_db_name = 'application'

connection.set_isolation_level(0)

with connection.cursor() as cursor:
    cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(new_db_name)))
connection.close()

In [35]:
# database parameters 
db_params = {
    'host': "desperado-db.ctldmj6kaxoc.us-east-2.rds.amazonaws.com",
    'port': '5432',
    'user': 'desperado',
    'password': '6156dbdesperado',
    'database': 'application'
}

In [92]:
# delete table 
with psycopg2.connect(**db_params) as connection:
    with connection.cursor() as cursor:
        # Drop the table
        cursor.execute(f"DROP TABLE IF EXISTS application;")

    # Commit the changes
    connection.commit()

In [93]:
# Define the table schema
table_schema = {
    'applicationID': 'VARCHAR(32) PRIMARY KEY', 
    'jobID': 'VARCHAR(255)',
    'applicantName': 'VARCHAR(255)',
    'applicantEmail': 'VARCHAR(255)',
    'recruiterName': 'VARCHAR(255)',
    'recruiterEmail': 'VARCHAR(255)',
    'status': 'VARCHAR(255)'
}

# Establish a connection to the PostgreSQL database
with psycopg2.connect(**db_params) as conn_app:
    with conn_app.cursor() as cursor:
        # Enable the uuid-ossp extension
        cursor.execute("CREATE EXTENSION IF NOT EXISTS \"uuid-ossp\";")

        # Create the table using the specified schema
        create_table_query = f"CREATE TABLE IF NOT EXISTS application ({', '.join(f'{col} {data_type}' for col, data_type in table_schema.items())});"
        cursor.execute(create_table_query)

    # Commit the changes
    conn_app.commit()

print("Table 'application' created successfully.")

Table 'application' created successfully.


In [94]:
# populate tabl with the csv file
csv_file_path = './application.csv'

# Establish a connection to the PostgreSQL database
with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cursor:
        # Read the CSV file
        with open(csv_file_path, 'r') as csvfile:
            csv_reader = csv.DictReader(csvfile)
            for row in csv_reader:                
                # Prepare the SQL query for insertion
                insert_query = f"INSERT INTO application ({', '.join(row.keys())}) VALUES ({', '.join(['%s'] * len(row))});"

                # Execute the query
                cursor.execute(insert_query, tuple(row.values()))

    # Commit the changes
    conn.commit()

print("Data inserted successfully.")

Data inserted successfully.


In [95]:
# sanity check - list 10 entries
with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cursor:
        cursor.execute("SELECT * FROM application LIMIT 10;")
        rows = cursor.fetchall()
        
        # Process the results
        if rows:
            print("Data in the table:")
            for row in rows:
                print(row)
        else:
            print("No data found in the table.")

Data in the table:
('6562df23fc13ae6b204cd964', '233', 'Jocelyn Sayre', 'jsayreo0@cbc.ca', 'Rafa Gammett', 'rgammettad@w3.org', 'pending')
('6562df23fc13ae6b204cd965', '579', 'Fanny Seagood', 'fseagoodh6@qq.com', "Hewett O'Luney", 'holuneyoj@noaa.gov', 'rejected')
('6562df23fc13ae6b204cd966', '501', 'Sileas Axleby', 'saxlebyh4@fc2.com', 'Ronny Massie', 'rmassiej8@phoca.cz', 'rejected')
('6562df23fc13ae6b204cd967', '799', 'Gweneth Scally', 'gscally6z@nyu.edu', 'Kermit Vanichkov', 'kvanichkovqh@webnode.com', 'rejected')
('6562df23fc13ae6b204cd968', '929', 'Georgianna Giannazzo', 'ggiannazzock@indiatimes.com', 'Elijah Eads', 'eeadsgs@blogspot.com', 'pending')
('6562df23fc13ae6b204cd969', '308', 'Lauritz Beverstock', 'lbeverstock1e@wunderground.com', 'Bron Templman', 'btemplmann@ucla.edu', 'rejected')
('6562df23fc13ae6b204cd96a', '823', 'Marcelline Wilflinger', 'mwilflingerbl@elegantthemes.com', 'Aube Willingale', 'awillingale5w@si.edu', 'accepted')
('6562df23fc13ae6b204cd96b', '30', 'Dina

In [89]:
# sanity check - get schema of the table
with psycopg2.connect(**db_params) as conn_app:
    with conn_app.cursor() as cursor:
        schema_query = sql.SQL("""
            SELECT column_name, data_type
            FROM information_schema.columns
            WHERE table_name = {}
        """).format(sql.Literal('application'))
        # Execute the query
        cursor.execute(schema_query)
        
        # Fetch and print the result
        schema = cursor.fetchall()
        print(f"Schema for table application:\n{schema}")

Schema for table application:
[('applicationid', 'character varying'), ('jobid', 'character varying'), ('applicantname', 'character varying'), ('applicantemail', 'character varying'), ('recruitername', 'character varying'), ('recruiteremail', 'character varying'), ('status', 'character varying')]


In [90]:
# list all tables
with psycopg2.connect(**db_params) as conn_app:
    with conn_app.cursor() as cursor:
        cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public' AND table_type='BASE TABLE';")
        tables = cursor.fetchall()
        for table in tables:
            print(table[0])

application


In [101]:
with psycopg2.connect(**db_params) as conn_app:
    connection.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_DEFAULT)

In [98]:
if cursor:
    cursor.close()
if conn_app:
    conn_app.close()